#UK trade in goods by industry, country and commodity

This data is split into two distributions, one for imports and the other for exports:
https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradeingoodsbyindustrycountryandcommodityimports

https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradeingoodsbyindustrycountryandcommodityexports

In [1]:
from gssutils import *

In [2]:
def run_script(s):
    %run "$s"
    return table

observations = pd.concat(
    run_script(s) for s in ['exports.ipynb', 'imports.ipynb']
).drop_duplicates()

'DATAMARKER'

Index(['suppressed data'], dtype='object')

'ONS Partner Geography'

Index(['AD Andorra', 'AE United Arab Emirates', 'AF Afghanistan',
       'AG Antigua & Barbuda', 'AI Anguilla', 'AL Albania', 'AM Armenia',
       'AO Angola', 'AQ Antarctica', 'AR Argentina',
       ...
       'VN Vietnam', 'VU Vanuatu', 'W1 Worldwide', 'WF Wallis & Futuna',
       'WS Samoa', 'XK Kosovo', 'YE Yemen', 'ZA South Africa', 'ZM Zambia',
       'ZW Zimbabwe'],
      dtype='object', length=235)

'CORD SITC'

...

'Flow'

Index(['imports'], dtype='object')

Index(['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '11', '12',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '32', '33', '34',
       '35', '41', '42', '43', '51', '52', '53', '54', '55', '56', '57', '58',
       '59', '61', '62', '63', '64', '65', '66', '67', '68', '69', '78', '79',
       '7E', '7M', '84', '85', '87', '8O', '9 ', 'T '],
      dtype='object')

Index(['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR',
       ...
       'VN', 'VU', 'W1', 'WF', 'WS', 'XK', 'YE', 'ZA', 'ZM', 'ZW'],
      dtype='object', length=235)

Index(['01', '02', '03', '05', '06', '07', '08', '09', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '35', '36', '37', '38',
       '39', '41', '42', '43', '45', '46', '47', '49', '50', '51', '52', '53',
       '55', '56', '58', '59', '60', '61', '62', '63', '64', '65', '66', '68',
       '69', '70', '71', '72', '73', '74', '75', '77', '78', '79', '80', '81',
       '82', '84', '85', '86', '87', '88', '90', '91', '92', '93', '94', '95',
       '96', '98', '99'],
      dtype='object')

In [3]:
from pathlib import Path
import numpy as np
out = Path('out')
out.mkdir(exist_ok=True)
observations.drop_duplicates().to_csv(out / 'observations.csv', index = False)

In [4]:
from gssutils.metadata import THEME
scraper.dataset.family = 'Trade'
scraper.dataset.theme = THEME['business-industry-trade-energy']
scraper.dataset.title = scraper.dataset.title.replace('imports', 'imports and exports')
scraper.dataset.comment = scraper.dataset.comment.replace('import', 'import and export')

scraper.dataset

In [5]:
with open(out / 'dataset.trig', 'wb') as metadata:
     metadata.write(scraper.generate_trig())